In [1]:
pip install fastestimator ~/repos/fledge

Processing /home/itrushkin/repos/fledge


  Created wheel for fledge: filename=fledge-0.1-py3-none-any.whl size=169008 sha256=ddf05bf28d8057572e51303dedea47afe946bc1ab77bc91e21ef78f213ae7aed
  Stored in directory: /tmp/pip-ephem-wheel-cache-wiwjix7k/wheels/3a/30/9d/a88101effd240e4a8721301bebf95a245eaf4904b55b7ae530
Successfully built fledge
  Attempting uninstall: fledge
    Found existing installation: fledge 0.1
    Uninstalling fledge-0.1:
      Successfully uninstalled fledge-0.1
You should consider upgrading via the '/home/itrushkin/.virtualenvs/fledge/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import fastestimator as fe
from fastestimator.architecture.pytorch import LeNet
from fastestimator.backend import to_tensor, argmax
from fastestimator.dataset.data import cifar10
from fastestimator.op.numpyop.meta import Sometimes
from fastestimator.op.numpyop.multivariate import HorizontalFlip
from fastestimator.op.numpyop.univariate import CoarseDropout, Normalize, Onehot
from fastestimator.op.tensorop import Average
from fastestimator.op.tensorop.gradient import Watch, FGSM
from fastestimator.op.tensorop.loss import CrossEntropy
from fastestimator.op.tensorop.model import ModelOp, UpdateOp
from fastestimator.trace.io import BestModelSaver
from fastestimator.trace.metric import Accuracy
from fastestimator.util import ImgData, to_number
from fledge.native.fastestimator import FederatedFastEstimator
from fastestimator.dataset.data import cifar10
from fastestimator.op.numpyop.univariate import Normalize, ChannelTranspose

#data_path='1'
#collaborator_count=2
batch_size=128

train_data, eval_data = cifar10.load_data()
test_data = eval_data.split(0.5)
#train_data, eval_data, test_data = split_data(train_data, eval_data, test_data, int(data_path), collaborator_count)

pipeline = fe.Pipeline(train_data=train_data,
                     eval_data=eval_data,
                     test_data=test_data,
                     batch_size=batch_size,
                     ops=[Normalize(inputs="x", outputs="x",
                             mean=(0.4914, 0.4822, 0.4465), 
                             std=(0.2471, 0.2435, 0.2616)),
                          ChannelTranspose(inputs="x", outputs="x")])
model = fe.build(model_fn=lambda: LeNet(input_shape=(3, 32, 32)), \
                         optimizer_fn="adam", model_name="adv_model")

network = fe.Network(ops=[
                Watch(inputs="x"),
                ModelOp(model=model, inputs="x", outputs="y_pred"),
                CrossEntropy(inputs=("y_pred", "y"), outputs="base_ce"),
                FGSM(data="x", loss="base_ce", outputs="x_adverse", epsilon=0.04),
                ModelOp(model=model, inputs="x_adverse", outputs="y_pred_adv"),
                CrossEntropy(inputs=("y_pred_adv", "y"), outputs="adv_ce"),
                Average(inputs=("base_ce", "adv_ce"), outputs="avg_ce"),
                UpdateOp(model=model, loss_name="avg_ce")
            ])

estimator = fe.Estimator(pipeline=pipeline,
                         network=network,
                         epochs=2,
                         traces=[Accuracy(true_key="y", pred_key="y_pred", output_name="clean_accuracy"),
                                Accuracy(true_key="y", pred_key="y_pred_adv", output_name="adversarial_accuracy")],
                         max_train_steps_per_epoch=None,
                         max_eval_steps_per_epoch=None,
                         monitor_names=["base_ce", "adv_ce"],
                         log_steps=1000)
fledge_estimator = FederatedFastEstimator(estimator, rounds=10)
model=fledge_estimator.fit()

Instructions for updating:
non-resource variables are not supported in the long term
Creating Workspace Directories
Creating Workspace Certifications
Creating Workspace Templates

New workspace directory structure:
workspace
├── code
├── agg_to_col_two_signed_cert.zip
├── plan
│   ├── cols.yaml
│   ├── data.yaml
│   ├── defaults
│   │   ├── tasks_keras.yaml
│   │   ├── tasks_tensorflow.yaml
│   │   ├── tasks_fast_estimator.yaml
│   │   ├── network.yaml
│   │   ├── collaborator.yaml
│   │   ├── assigner.yaml
│   │   ├── tasks_torch.yaml
│   │   ├── aggregator.yaml
│   │   ├── task_runner.yaml
│   │   └── data_loader.yaml
│   └── plan.yaml
├── requirements.txt
├── logs
├── agg_to_col_one_signed_cert.zip
├── cert
│   ├── config
│   │   ├── client.conf
│   │   ├── signing-ca.conf
│   │   ├── server.conf
│   │   └── root-ca.conf
│   ├── client
│   │   ├── col_one.crt
│   │   ├── col_two.csr
│   │   ├── col_two.crt
│   │   ├── col_two.key
│   │   ├── col_one.csr
│   │   └── col_one.key
│   ├

/home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/federated/task/runner_pt.py:172: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  new_state[k] = pt.from_numpy(tensor_dict.pop(k)).to(device)


FastEstimator-Test: step: None; epoch: 2; clean_accuracy: 0.0952; adversarial_accuracy: 0.0494; 


    ______           __  ______     __  _                 __            
   / ____/___ ______/ /_/ ____/____/ /_(_)___ ___  ____ _/ /_____  _____
  / /_  / __ `/ ___/ __/ __/ / ___/ __/ / __ `__ \/ __ `/ __/ __ \/ ___/
 / __/ / /_/ (__  ) /_/ /___(__  ) /_/ / / / / / / /_/ / /_/ /_/ / /    
/_/    \__,_/____/\__/_____/____/\__/_/_/ /_/ /_/\__,_/\__/\____/_/     
                                                                        

FastEstimator-Warn: No ModelSaver Trace detected. Models will not be saved.
FastEstimator-Start: step: 1; num_device: 3; logging_interval: 1000; 
FastEstimator-Train: step: 1; base_ce: 2.2967572; avg_ce: 2.3071933; adv_ce: 2.317629; 
FastEstimator-Train: step: 391; epoch: 1; epoch_time: 11.11 sec; 
FastEstimator-Eval: step: 391; epoch: 1; base_ce: 1.3600664; avg_ce: 1.5109972; adv_ce: 1.6619282; clean_accuracy: 0.5072; adversarial_accuracy: 0.3714; 
FastEstimator-Train: step: 782; epoch: 2; epoch_time: 11.18 sec; 
FastEstimator-Eval: step: 782; epoch: 2; 

FastEstimator-Test: step: 782; epoch: 2; clean_accuracy: 0.5802; adversarial_accuracy: 0.412; 


...

    ______           __  ______     __  _                 __            
   / ____/___ ______/ /_/ ____/____/ /_(_)___ ___  ____ _/ /_____  _____
  / /_  / __ `/ ___/ __/ __/ / ___/ __/ / __ `__ \/ __ `/ __/ __ \/ ___/
 / __/ / /_/ (__  ) /_/ /___(__  ) /_/ / / / / / / /_/ / /_/ /_/ / /    
/_/    \__,_/____/\__/_____/____/\__/_/_/ /_/ /_/\__,_/\__/\____/_/     
                                                                        

FastEstimator-Warn: No ModelSaver Trace detected. Models will not be saved.
FastEstimator-Start: step: 1; num_device: 3; logging_interval: 1000; 
FastEstimator-Train: step: 1; base_ce: 0.4452236; avg_ce: 0.682985; adv_ce: 0.92074645; 
FastEstimator-Train: step: 391; epoch: 1; epoch_time: 11.21 sec; 
FastEstimator-Eval: step: 391; epoch: 1; base_ce: 0.79762715; avg_ce: 1.1418564; adv_ce: 1.4860859; clean_accuracy: 0.732; adversarial_accuracy: 0.4942; 
FastEstimator-Train: step: 782; epoch: 2; epoch_time: 11.08 sec; 
FastEstimator-Eval: step: 782; epoch: 2;

FastEstimator-Test: step: 782; epoch: 2; clean_accuracy: 0.7244; adversarial_accuracy: 0.484; 


FastEstimator-Test: step: 782; epoch: 2; clean_accuracy: 0.7274; adversarial_accuracy: 0.4914; 


    ______           __  ______     __  _                 __            
   / ____/___ ______/ /_/ ____/____/ /_(_)___ ___  ____ _/ /_____  _____
  / /_  / __ `/ ___/ __/ __/ / ___/ __/ / __ `__ \/ __ `/ __/ __ \/ ___/
 / __/ / /_/ (__  ) /_/ /___(__  ) /_/ / / / / / / /_/ / /_/ /_/ / /    
/_/    \__,_/____/\__/_____/____/\__/_/_/ /_/ /_/\__,_/\__/\____/_/     
                                                                        

FastEstimator-Warn: No ModelSaver Trace detected. Models will not be saved.
FastEstimator-Start: step: 1; num_device: 3; logging_interval: 1000; 
FastEstimator-Train: step: 1; base_ce: 0.47057852; avg_ce: 0.72819483; adv_ce: 0.9858111; 
FastEstimator-Train: step: 391; epoch: 1; epoch_time: 11.13 sec; 
FastEstimator-Eval: step: 391; epoch: 1; base_ce: 0.7968623; avg_ce: 1.141045; adv_ce: 1.4852278; clean_accuracy: 0.7302; adversarial_accuracy: 0.4938; 
FastEstimator-Train: step: 782; epoch: 2; epoch_time: 11.05 sec; 
FastEstimator-Eval: step: 782; epoch: 2

FastEstimator-Test: step: 782; epoch: 2; clean_accuracy: 0.7244; adversarial_accuracy: 0.4872; 


In [3]:
model.state_dict()

OrderedDict([('module.conv1.weight',
              tensor([[[[ 1.0045e-01, -1.6815e-02, -5.0068e-02],
                        [ 1.4253e-01, -5.5407e-02, -5.8111e-02],
                        [ 1.7893e-01, -1.0924e-01, -3.5104e-02]],
              
                       [[ 9.6488e-02,  4.0903e-02, -9.2665e-03],
                        [ 1.4534e-01,  3.3700e-02, -4.7639e-02],
                        [ 1.6572e-01, -5.9179e-02, -8.1506e-03]],
              
                       [[-6.8587e-03, -6.1685e-02, -7.2202e-02],
                        [-1.4615e-02, -9.3403e-02, -7.1748e-02],
                        [ 2.0029e-02, -1.4099e-01, -6.5369e-02]]],
              
              
                      [[[ 6.4476e-02,  1.1921e-01,  2.9656e-02],
                        [ 2.2803e-01,  2.5284e-01,  2.0191e-01],
                        [ 1.8031e-01,  2.6462e-01,  1.5604e-01]],
              
                       [[-6.2902e-02, -7.6621e-02, -3.4148e-02],
                        [-1.2571e-01, 